In [1]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.ml import PipelineModel

In [2]:
# input and output folders
trainingData = "ex_data/trainingData.csv"
unlabeledData = "ex_data/unlabeledData.csv"
outputPath = "predictionsDTPipeline/"

In [3]:
# *************************
# Training step
# *************************

# Create a DataFrame from trainingData.csv
# Training data in raw format
trainingData = spark.read.load(trainingData,\
                     format="csv",\
                     header=True,\
                     inferSchema=True)

In [4]:
#trainingData.show()

In [5]:
# Define an assembler to create a column (features) of type Vector 
# containing the double values associated with columns attr1, attr2, attr3
assembler = VectorAssembler(inputCols=["attr1", "attr2", "attr3"],\
                            outputCol="features")

In [6]:
# Create a DecisionTreeClassifier object.  
# DecisionTreeClassifier is an Estimator that is used to 
# create a classification model based on decision trees.
dt = DecisionTreeClassifier()

In [7]:
# We can set the values of the parameters of the Decision Tree
# For example we can set the measure that is used to decide if a 
# node must be split.
# In this case we set gini index
dt.setImpurity("gini")

DecisionTreeClassifier_09966ca04b21

In [8]:
# Define a pipeline that is used to create the decision tree
# model on the training data. The pipeline includes also 
# the preprocessing step
pipeline = Pipeline().setStages([assembler, dt])

In [9]:
# Execute the pipeline on the training data to build the 
# classification model
classificationModel = pipeline.fit(trainingData)

In [10]:
# Now, the classification model can be used to predict the class label
# of new unlabeled data

In [11]:
# Create a DataFrame from unlabeledData.csv
# Unlabeled data in raw format
unlabeledData = spark.read.load(unlabeledData,\
                     format="csv", header=True, inferSchema=True)

In [12]:
#unlabeledData.printSchema()
#unlabeledData.show()

In [13]:
# Make predictions on the unlabled data using the transform() method of the 
# trained classification model transform uses only the content of 'features' 
# to perform the predictions. The model is associated with the pipeline and hence
# also the assembler is executed
predictionsDF = classificationModel.transform(unlabeledData)

In [14]:
#predictionsDF.printSchema()
#predictionsDF.show()

In [15]:
# The returned DataFrame has the following schema (attributes)
# - features: vector (values of the attributes)
# - label: double (value of the class label)
# - rawPrediction: vector (nullable = true)
# - probability: vector (The i-th cell contains the probability that the current 
#	record belongs to the i-th class
# - prediction: double (the predicted class label)

# Select only the original features (i.e., the value of the original attributes 
# attr1, attr2, attr3) and the predicted class for each record
predictions = predictionsDF.select("attr1", "attr2", "attr3", "prediction")

In [16]:
# Save the result in an HDFS output folder
predictions.write.csv(outputPath, header="true")